## 本地数据库方式

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_milvus import Milvus

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)
splits = text_splitter.split_documents(docs)
print(len(splits))

# 文本嵌入
embed = ZhipuAIEmbeddings(model="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 向量库创建 - 添加 index_params to specify supported index type
vectorstore = Milvus.from_documents(
    documents=splits,  # You should use the splits, not the original docs
    embedding=embed,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    index_params={
        "metric_type": "L2",  # or "IP" depending on your needs
        "index_type": "IVF_FLAT",  # Use a supported index type
        "params": {"nlist": 128}  # IVF_FLAT parameter
    },
    drop_old=True,
)


# 检索
question = "图像识别"
docs = vectorstore.similarity_search(question,k=3)
print(len(docs))
print(docs[0].page_content)

## 远端数据库方式

In [ ]:
!docker port milvus-standalone 19530/tcp

In [ ]:
from pymilvus import connections, db
 
conn = connections.connect(host="129.201.70.31", port=19530)
database = db.create_database("sample_db")

In [ ]:
测试

In [ ]:
from pymilvus import (
    db,
    MilvusClient,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

#1.创建Milvus客户端
fmt = "\n=== {:30} ===\n"
# 1. connect to Milvus 数据库必须选存在，可通过可视化的管理界面创建数据库
print(fmt.format("1. start connecting to Milvus"))
milvusclient = MilvusClient(uri="http://120.79.252.32:19530", db_name="default")

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530"
)

client.list_databases()


# client.describe_database(
#     db_name="default"
# )



In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=(
        "https://lilianweng.github.io/posts/2023-06-23-agent/",
        "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

docs = text_splitter.split_documents(documents)

docs[1]


In [ ]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import ZhipuAIEmbeddings

embed = ZhipuAIEmbeddings(model="embedding-2",api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")
vector = Milvus.from_documents(
     documents=documents, # 设置保存的文档
     embedding=embed, # 设置 embedding model
     collection_name="book2", # 设置 集合名称
     drop_old=True,
     connection_args={"host": "120.79.252.32", "port": "19530", "db_name":"sample_db"},# Milvus连接配置
)


In [ ]:
query = "What is self-reflection of an AI Agent?"
vector.similarity_search(query, k=1)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_milvus import Milvus
# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)
print(len(splits))

# 文本嵌入
embed = ZhipuAIEmbeddings(model="embedding-2",api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 测试
# text_1 = "今天天气不错"

# query_result = embeddings.embed_query(text_1)
# print(query_result)

# 路径设置
# persist_directory = './langchain-milvus'

# 向量库创建
connection_args = {
    "host": "129.201.70.31",
    "port": "19530",
}
vectordb = Milvus.from_documents(
    documents=docs,
    embedding=embed,
    collection_name="book1",
    drop_old=False,
    connection_args=connection_args,
)
# print(vectordb._collection.count())

# 检索
question = "图像识别"
docs = vectordb.similarity_search(question,k=3)
print(len(docs))
print(docs[0].page_content)

In [ ]:
# 